In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver # this is used to save the context of the previous chat

from typing import TypedDict, Annotated

import os
from dotenv import load_dotenv

/Users/consultadd/Prajwal/consultadd_langGraph/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()

llm= HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    huggingfacehub_api_token= os.getenv("HUGGINGFACE_API_ENDPOINT"),
    max_new_tokens= 500,
    temperature= 0.3
)

model= ChatHuggingFace(llm= llm)

/Users/consultadd/Prajwal/consultadd_langGraph/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
def chat_node(state:ChatState)-> ChatState:
    # take the user query from the state
    messages= state['messages']
    
    # send to llm
    response= model.invoke(messages)
    
    # store the response of the state
    return {"messages": [response]}

In [5]:
checkpointer= MemorySaver()

graph= StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot= graph.compile(checkpointer= checkpointer)

In [ ]:
initial_state= {
    'messages':[HumanMessage(content="What is the capital of India")]
}
# final_state= chatbot.get_state(config= )

SyntaxError: invalid syntax (3226357244.py, line 4)

In [ ]:
# final_state

{'messages': [HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}, id='fd774b0d-2036-4f4f-84b2-18a71e9f60b2'),
  AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 41, 'total_tokens': 50}, 'model_name': 'meta-llama/Llama-3.2-3B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--610b4017-d765-47a0-9ba1-3230d9371325-0', usage_metadata={'input_tokens': 41, 'output_tokens': 9, 'total_tokens': 50})]}

In [7]:
thread_id= '1'


while True:
    user_message= input("Type here: ")
    
    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break
    
    config= {'configurable': {'thread_id': thread_id}}
    response= chatbot.invoke({'message': [HumanMessage(content=user_message)]}, config= config)
    
    print('AI: ', response['messages'][-1].content)
    

KeyboardInterrupt: 